## MoE的过程

1. 用$ x $表示为包含$ T $个token的输入（$ T=\text{batchsize}*\text{seq\_len} $），$ x \in \mathbb{R}^{T \times d} $，$ W $表示门控权重矩阵，$ b $表示门控偏置向量，获得logits $ z \sim \mathbb{R}^{T \times N} $（$ N $是专家的个数）。
$$
z(x) = xW + b
$$
2. 由logit获得路由概率$ s_{i,j}(x) $，表示第$ i $个token选择第$ j $个专家的路由概率（亲和度得分）。
$$
s_{i,j}(x) = \text{softmax}(z_{i,j}(x)) = \frac{\exp(z_{i,j}(x))}{\sum_{k=1}^{N} \exp(z_{i,k}(x))}
$$
3. Top - $ k $选择，选出第$ i $个token的前$ K $高概率的专家：
$$
g_{i,j}(x) =
\begin{cases}
s_{i,j}(x) & \text{if } s_{i,j}(x) \in \text{TopK}(\{s_{i,j}(x)\}_{j = 1}^{N}, K) \\
0 & \text{otherwise}
\end{cases}
$$
4. 对选出的$ K $个专家重新归一化：
$$
v_{i,j}(x) = \frac{g_{i,j}(x)}{\sum_{k = 1}^{N} g_{i,k}(x)}
$$
5. 对于第$ i $个token的MoE输出为：
$$
\text{MoE}(x_i) = \sum_{j = 1}^{N} v_{i,j}(x) \cdot \text{Expert}(x_i)
$$

## MoE代码实现

> 参考实现路径: `transformers/src/transformers/models/qwen3_moe/modular_qwen3_moe.py#L67`



具体的代码实现分为两个阶段：
1. **路由阶段**：
- 对每个token，计算所有专家的权重（router_logits）
- 选出top_k个权重最大的专家，同时记录哪些专家被至少一个token选中（expert_hit）

2. **专家计算阶段**：
- 对每个被选中的专家（expert_hit）：
a. 收集所有选择该专家的token
b. 将这些token的输入拼接起来（或保持独立）
c. 通过该专家网络计算输出
d. 用路由权重对输出进行加权



In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from icecream import ic

# 简化的专家网络定义
class BasicExpert(nn.Module):
    # 一个 Expert 可以是一个最简单的 linear 层
    # 也可以是 MLP 层
    # 也可以是更复杂的 MLP 层（active function 设置为 swiglu）
    def __init__(self, feature_in, feature_out):
        super().__init__()
        self.linear = nn.Linear(feature_in, feature_out)
    
    def forward(self, x):
        return self.linear(x)

class Qwen3MoeSparseMoeBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_experts = config.num_experts  # 专家网络数量
        self.top_k = config.num_experts_per_tok  # 每个token使用的专家数
        self.norm_topk_prob = config.norm_topk_prob  # 是否对top_k权重进行归一化

        # 门控网络：决定每个token应该被分配到哪个专家
        self.gate = nn.Linear(config.hidden_size, config.num_experts, bias=False)
        # 创建多个专家网络（使用简化的BasicExpert）
        self.experts = nn.ModuleList(
            [BasicExpert(config.hidden_size, config.hidden_size) for _ in range(self.num_experts)]
        )

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        """稀疏混合专家模型前向传播"""
        batch_size, sequence_length, hidden_dim = hidden_states.shape
        # 将输入展平: (batch_size, sequence_length, hidden_dim) -> (batch_size*sequence_length, hidden_dim)
        hidden_states = hidden_states.view(-1, hidden_dim)
        
        # 计算路由logits: (batch*sequence_length, n_experts)
        router_logits = self.gate(hidden_states)

        # 通过softmax计算每个专家对每个token的权重
        routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
        # 选择权重最高的top_k个专家
        routing_weights, selected_experts = torch.topk(routing_weights, self.top_k, dim=-1)
        
        # 如果启用归一化，对top_k权重进行归一化
        if self.norm_topk_prob:
            routing_weights /= routing_weights.sum(dim=-1, keepdim=True)
        
        # 将权重转换回输入的数据类型
        routing_weights = routing_weights.to(hidden_states.dtype)

        # 初始化最终输出张量
        final_hidden_states = torch.zeros(
            (batch_size * sequence_length, hidden_dim), dtype=hidden_states.dtype, device=hidden_states.device
        )

        # 创建专家掩码：使用one-hot编码标记每个token选择了哪些专家
        # 形状: (n_experts, top_k, batch*sequence_length)
        expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=self.num_experts).permute(2, 1, 0)
        ic(expert_mask.shape)
        ic(expert_mask)

        # 找出至少被一个token选中的专家（活跃专家）
        expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
        
        # 遍历所有被激活的专家
        for expert_idx in expert_hit:
            expert_layer = self.experts[expert_idx]  # 获取当前专家网络
            
            # 找出选择当前专家的所有token及其在top_k中的位置
            idx, top_x = torch.where(expert_mask[expert_idx].squeeze(0))

            # 提取对应token的隐藏状态并通过专家网络处理
            current_state = hidden_states[None, top_x].reshape(-1, hidden_dim)
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]

            # 将当前专家的计算结果累加到最终输出中
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
        
        # 恢复输出形状: (batch_size*sequence_length, hidden_dim) -> (batch_size, sequence_length, hidden_dim)
        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)
        return final_hidden_states, router_logits  # 返回输出和路由logits

# 创建一个简单的配置类用于测试
class SimpleConfig:
    def __init__(self):
        self.num_experts = 4
        self.num_experts_per_tok = 2
        self.norm_topk_prob = True
        self.hidden_size = 64

# 测试代码
if __name__ == "__main__":
    # 创建配置和模型
    config = SimpleConfig()
    model = Qwen3MoeSparseMoeBlock(config)
    
    # 生成随机输入数据
    batch_size, seq_len, hidden_size = 2, 5, config.hidden_size
    dummy_input = torch.randn(batch_size, seq_len, hidden_size)
    
    print("输入形状:", dummy_input.shape)
    
    # 前向传播
    output, router_logits = model(dummy_input)
    
    print("输出形状:", output.shape)
    print("路由器logits形状:", router_logits.shape)
    print("前向传播完成!")

ic| expert_mask.shape: torch.Size([4, 2, 10])
ic| expert_mask: tensor([[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]],
                 
                         [[0, 0, 1, 0, 1, 1, 0, 0, 1, 0],
                          [1, 0, 0, 0, 0, 0, 0, 1, 0, 1]],
                 
                         [[1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
                          [0, 1, 1, 0, 1, 1, 0, 0, 0, 0]],
                 
                         [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])


输入形状: torch.Size([2, 5, 64])
输出形状: torch.Size([2, 5, 64])
路由器logits形状: torch.Size([10, 4])
前向传播完成!
